# Feature selection

**Edit: I realized that this model is practically useless since I forgot to include lag the features, and the price transform features are of course highly correlated to the price, hence the R^2 = 1.**

Here I will test which features are best to train the models with since there's so many.  
I will do this using Linear Regression and Mean Squared Error.  
I test on the trading pair ADA/USDT for different timeframes, the common ones: 1m, 5m, 15m, 30m, 45m, 1h, 2h, 4h, 1d

In [5]:
import pandas as pd
import talib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [6]:
df = pd.read_csv("../Data/ADA/ADA_USDT_1d.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

df.head(5)

,open,high,low,close,volume
datetime,,,,,
2021-10-22,2.141,2.208,2.133,2.158,121842.2
2021-10-23,2.158,2.180,2.141,2.157,103037.4
2021-10-24,2.157,2.162,2.100,2.121,131466.3
2021-10-25,2.121,2.182,2.116,2.152,188189.8
2021-10-26,2.152,2.205,2.130,2.136,255700.5


In [7]:
# Technical indicators - used ChatGPT to add all of them to the dataset. Later, I will optimize to see which ones are best.

# Overlap Studies
df['DEMA'] = talib.DEMA(df['close'], timeperiod=30)
df['EMA'] = talib.EMA(df['close'], timeperiod=30)
df['KAMA'] = talib.KAMA(df['close'], timeperiod=30)
df['MA'] = talib.MA(df['close'], timeperiod=30)
df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0.5, slowlimit=0.05)
df['MIDPOINT'] = talib.MIDPOINT(df['close'], timeperiod=14)
df['MIDPRICE'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=14)
df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
df['SAREXT'] = talib.SAREXT(df['high'], df['low'], startvalue=0, offsetonreverse=0, accelerationinitlong=0.02,
                            accelerationlong=0.02, accelerationmaxlong=0.2, accelerationinitshort=0.02,
                            accelerationshort=0.02, accelerationmaxshort=0.2)
df['SMA'] = talib.SMA(df['close'], timeperiod=30)
df['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0.7)
df['TEMA'] = talib.TEMA(df['close'], timeperiod=30)
df['TRIMA'] = talib.TRIMA(df['close'], timeperiod=30)
df['WMA'] = talib.WMA(df['close'], timeperiod=30)

# Momentum Indicators
df['ADX'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
df['ADXR'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=14)
df['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26, matype=0)
df['AROON_DOWN'], df['AROON_UP'] = talib.AROON(df['high'], df['low'], timeperiod=14)
df['AROONOSC'] = talib.AROONOSC(df['high'], df['low'], timeperiod=14)
df['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
df['CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
df['CMO'] = talib.CMO(df['close'], timeperiod=14)
df['DX'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=14)
df['MACD'], df['MACDSIGNAL'], df['MACDHIST'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['MACDEXT'], df['MACDSIGNALEXT'], df['MACDHISTEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26,
                                                                      slowmatype=0, signalperiod=9, signalmatype=0)
df['MACDFIX'], df['MACDSIGNALFIX'], df['MACDHISTFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
df['MFI'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
df['MINUS_DI'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
df['MINUS_DM'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=14)
df['MOM'] = talib.MOM(df['close'], timeperiod=10)
df['PLUS_DI'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
df['PLUS_DM'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=14)
df['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
df['ROC'] = talib.ROC(df['close'], timeperiod=10)
df['ROCP'] = talib.ROCP(df['close'], timeperiod=10)
df['ROCR'] = talib.ROCR(df['close'], timeperiod=10)
df['ROCR100'] = talib.ROCR100(df['close'], timeperiod=10)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)
df['STOCH'], df['STOCH_SLOWD'] = talib.STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df['STOCHF'], df['STOCHF_FASTD'] = talib.STOCHF(df['high'], df['low'], df['close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df['STOCHRSI'], df['STOCHRSI_FASTD'] = talib.STOCHRSI(df['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
df['TRIX'] = talib.TRIX(df['close'], timeperiod=30)
df['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=14)

# Volume Indicators
df['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
df['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
df['OBV'] = talib.OBV(df['close'], df['volume'])

# Volatility Indicators
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
df['NATR'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=14)
df['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])

# Price Transform
df['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
df['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
df['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
df['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])

# Cycle Indicators
df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
df['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
df['HT_PHASOR_INPHASE'], df['HT_PHASOR_QUADRATURE'] = talib.HT_PHASOR(df['close'])
df['HT_SINE'], df['HT_LEADSINE'] = talib.HT_SINE(df['close'])
df['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])

# Statistical Functions
df['BETA'] = talib.BETA(df['high'], df['low'], timeperiod=5)
df['CORREL'] = talib.CORREL(df['high'], df['low'], timeperiod=30)
df['LINEARREG'] = talib.LINEARREG(df['close'], timeperiod=14)  # Linear Regression
df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'], timeperiod=14)
df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'], timeperiod=14)
df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'], timeperiod=14)
df['STDDEV'] = talib.STDDEV(df['close'], timeperiod=5, nbdev=1)
df['TSF'] = talib.TSF(df['close'], timeperiod=14)
df['VAR'] = talib.VAR(df['close'], timeperiod=5, nbdev=1)

# Bands
df['BBANDS_UPPER'], df['BBANDS_MIDDLE'], df['BBANDS_LOWER'] = talib.BBANDS(df['close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

df.dropna(inplace=True)

X = df.drop(columns=["close"])
y = df['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model_performance = []

# Starting point, all features
current_features = list(X_train.columns)

while current_features:
    # Model
    model = LinearRegression()
    model.fit(X_train[current_features], y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test[current_features])
    mse = mean_squared_error(y_test, y_pred)
    
    # Performance and current feature set
    model_performance.append((mse, current_features.copy()))
    
    if len(current_features) == 1:  # Stop if only one feature is left
        break
    
    feature_importances = np.abs(model.coef_)
    least_important_feature_index = np.argmin(feature_importances)
    least_important_feature = current_features[least_important_feature_index]
    
    # Remove least important feature
    current_features.remove(least_important_feature)

# Select best feature set based on performance
best_mse, best_features = min(model_performance, key=lambda x: x[0])

print(f"Best MSE: {best_mse}")
print(f"Best feature set: {best_features}")

Best MSE: 2.5439296958748785e-06
Best feature set: ['open', 'high', 'low', 'DEMA', 'EMA', 'KAMA', 'MA', 'MAMA', 'FAMA', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'APO', 'BOP', 'MACD', 'MACDSIGNAL', 'MACDHIST', 'MACDEXT', 'MACDSIGNALEXT', 'MACDHISTEXT', 'MACDFIX', 'MACDSIGNALFIX', 'MACDHISTFIX', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DM', 'PPO', 'STOCHF', 'STOCHF_FASTD', 'TRIX', 'ATR', 'NATR', 'TRANGE', 'HT_PHASOR_INPHASE', 'HT_PHASOR_QUADRATURE', 'HT_SINE', 'HT_LEADSINE', 'HT_TRENDMODE', 'BETA', 'CORREL', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR', 'BBANDS_UPPER', 'BBANDS_MIDDLE', 'BBANDS_LOWER']


## Results

**1m timeframe:**  
Best MSE: 4.1926485186385063e-32   
Best feature set: ['open', 'high', 'low', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE', 'BBANDS_UPPER', 'BBANDS_MIDDLE', 'BBANDS_LOWER']  

**5m timeframe:**  
Best MSE: 1.0186626769047126e-31  
Best feature set: ['high', 'low', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**15m timeframe:**
Best MSE: 6.414993420709901e-32  
Best feature set: ['high', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**30m timeframe:**  
Best MSE: 1.007609420543028e-31  
Best feature set: ['open', 'high', 'low', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**45m timeframe:**  
Best MSE: 8.64226907035605e-32  
Best feature set: ['high', 'low', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**1h timeframe result:**  
Best MSE: 9.387566886821251e-32  
Best feature set: ['open', 'high', 'low', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE', 'LINEARREG', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'TSF']  

**2h timeframe result:**  
Best MSE: 2.895685054205296e-32  
Best feature set: ['open', 'high', 'low', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**4h timeframe result:**  
Best MSE: 1.0722048820920871e-32  
Best feature set: ['open', 'high', 'low', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE']  

**1d timeframe result:**  
Best MSE: 7.131443451216737e-33  
Best feature set: ['MEDPRICE', 'WCLPRICE']  

## Analysis

The results are quite interesting. It is clear that features related to price movement has the most **predictive power**. This is perhaps not so surprising, as the price movement is correlated to the price, and technical indicators are typically used to provide insights into market behavior. The MSE score was also extremely low, suggesting high accuracy, though this may also be because of overfitting.